# Session 2 - Machine Learning for Time Series Data

This notebook mainly focuses on Time Series Analysis. It is built from the [Rossmann Store Sales](https://www.kaggle.com/c/rossmann-store-sales) competition hosted at Kaggle. We will explore the data and try to predict weeks of daily sales ($) for some of the stores. The forecast granularity or Level of Detail is Store and Week.

## Contents
[1. Exploratory Data Analysis](#1.-Exploratory-Data-Analysis)

[2. Time Series Components](#2.-Time-Series-Components)

[3. Statistical Model Forecasting](#3.-Statistical-Model-Forecasting)

[4. Machine Learning Forecasting](#4.-Machine-Learning-Forecasting)

<br>
## 1. Exploratory Data Analysis

As it usually goes, we start with the Exploratory Data Analysis of the main metrics revealing present trends and patterns in the data, giving a solid foundation for the further causal analysis.

### 1.1 Initial Load

Start by loading some packages

In [ ]:
import logging

from timeit import default_timer as timer

import warnings
warnings.filterwarnings("ignore")

# loading packages
# basic + dates 
import numpy as np
import pandas as pd
from pandas import datetime

# data visualization
import matplotlib.pyplot as plt
import seaborn as sns # advanced vizs
%matplotlib inline

# statistics
from statsmodels.distributions.empirical_distribution import ECDF

# time series analysis
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# prophet by Facebook
from fbprophet import Prophet
from fbprophet.diagnostics import cross_validation
from fbprophet.diagnostics import performance_metrics

# machine learning: XGB
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from xgboost.sklearn import XGBRegressor # wrapper

Load the **train** and **store** datasets. **train** is the training dataset with daily sales by product and store. **store** contains details about the stores.

In [ ]:
# importing train data to learn
train = pd.read_csv("data/train.csv", 
                    parse_dates = True, low_memory = False, index_col = 'Date')

# additional store data
store = pd.read_csv("data/store.csv", 
                    low_memory = False)

# Keep index as column as it will be used in the Machine Learning approach
train['Date'] = train.index

### 1.2 Initial Data Discovery

In this first section we go through the train and store data, handle missing values and create new columns for further analysis.

In [ ]:
# first glance at the train set: head and tail
print("In total: ", train.shape)
train.head(5).append(train.tail(5))

Short description of the **train** dataset:
- `Sales`: the turnover for any given day (target variable).
- `Customers`: the number of customers on a given day.
- `Open`: an indicator for whether the store was open: 0 = closed, 1 = open.
- `Promo`: indicates whether a store is running a promo on that day.
- `StateHoliday`: indicates a state holiday. Normally all stores, with few exceptions, are closed on state holidays. 
- `SchoolHoliday`: indicates if the (Store, Date) was affected by the closure of public schools.

### 1.3 ECDF (Empirical cumulative distribution function)

We are dealing with time series data so it will probably serve us to extract dates for further analysis. We also have two likely correlated vaiables in the dataset, which can be combined into a new feature.

In [ ]:
# data extraction
train['Year'] = train.index.year
train['Month'] = train.index.month
train['Day'] = train.index.day
train['WeekOfYear'] = train.index.weekofyear

# adding new variable
train['SalePerCustomer'] = train['Sales']/train['Customers']
train['SalePerCustomer'].describe()

On average customers spend about **$9.50 per day**. Though there are days with Sales equal to zero.

To get the first impression about continious variables in the data we can plot ECDF.

In [ ]:
sns.set(style = "ticks")# to format into seaborn 
c = '#386B7F' # basic color for plots
plt.figure(figsize = (12, 6))

plt.subplot(311)
cdf = ECDF(train['Sales'])
plt.plot(cdf.x, cdf.y, label = "statmodels", color = c);
plt.xlabel('Sales'); plt.ylabel('ECDF');

# plot second ECDF  
plt.subplot(312)
cdf = ECDF(train['Customers'])
plt.plot(cdf.x, cdf.y, label = "statmodels", color = c);
plt.xlabel('Customers');

# plot second ECDF  
plt.subplot(313)
cdf = ECDF(train['SalePerCustomer'])
plt.plot(cdf.x, cdf.y, label = "statmodels", color = c);
plt.xlabel('Sale per Customer');

About **20% of data has zero amount of sales/customers** that we need to deal with and almost 80% of time daily amount of sales was less than 1000. So what about zero sales, is it only due to the fact that the store is closed?

### 1.4 Missing values 
#### Closed stores with zero sales stores

In [ ]:
# closed stores
zero_sales_closed = train[(train.Open == 0) & (train.Sales == 0)]
print("In total: ", zero_sales_closed.shape)
zero_sales_closed.head(5)

There're **172,817 closed stores** in the data. It is about **10% of the total # of observations**. To avoid any biased forecasts we will drop these values. 

What about opened stores with zero sales?

In [ ]:
# opened stores with zero sales
zero_sales = train[(train.Open != 0) & (train.Sales == 0)]
print("In total: ", zero_sales.shape)
zero_sales.head(5)

Interestingly enough, there are opened store with __no sales on working days__. There're only 54 days in the data, so we can assume that there were external factors involved, for example manifestations, so we will also remove those observations.

In [ ]:
print("Closed stores and days which didn't have any sales won't be counted into the forecasts.")
train = train[(train["Open"] != 0) & (train['Sales'] != 0)]

print("In total: ", train.shape)

<div class = "alert alert-block alert-info"> It’s important to make a distinction between “Sales” and “Demand”. Sales is subject to available inventory and demand its not, demand its generally unconstrained. In practice it is very difficult to observe demand directly, so we use sales as proxy for demand. This has a problem because it doesn't account for situations where a customer wanted to purchase a product but it was unavailable (it will show up as zero sales). There are several imputation techniques to replace those cases with potential demand. </div>

What about **store** information:

In [ ]:
# additional information about the stores
store.head()

- `Store`: a unique Id for each store
- `StoreType`: differentiates between 4 different store models: a, b, c, d
- `Assortment`: describes an assortment level: a = basic, b = extra, c = extended
- `CompetitionDistance`: distance in meters to the nearest competitor store
- `CompetitionOpenSince[Month/Year]`: gives the approximate year and month of the time the nearest competitor was opened
- `Promo2`: Promo2 is a continuing a promotion for some stores: 0 = store is not participating, 1 = store is participating
- `Promo2Since[Year/Week]`: describes the year and calendar week when the store started participating in Promo2
- `PromoInterval`: describes the consecutive intervals Promo2 is started, naming the months the promotion is started. E.g. "Feb,May,Aug,Nov" means each round starts in February, May, August, November of any given year for that store

How many stores do we have?

In [ ]:
store['Store'].nunique()

Any missing values?

In [ ]:
store.isnull().sum()

We have few variables with missing values that we need to deal with. Let's start with the `CompetitionDistance`.

In [ ]:
# missing values in CompetitionDistance
store[pd.isnull(store.CompetitionDistance)]

Apperently this information is simply missing from the data. No particular pattern observed. In this case, it makes a complete sense to replace NaN with the median values (which is twice less that the average).

In [ ]:
# fill NaN with a median value (skewed distribuion)
store['CompetitionDistance'].fillna(store['CompetitionDistance'].median(), inplace = True)

Continuing further with missing data. What about `Promo2SinceWeek`? May it be that we observe unsusual data points?

In [ ]:
# no promo = no information about the promo?
_ = store[pd.isnull(store.Promo2SinceWeek)]
_[_.Promo2 != 0].shape

No, if there's no `Promo2` then there's no information about it. We can replace these values by zeros. The same goes for tha variables deducted from the competition, `CompetitionOpenSinceMonth` and `CompetitionOpenSinceYear`.

In [ ]:
# replace NA's by 0
store.fillna(0, inplace = True)

Lets join the **train** and **store** dataset for future modeling

In [ ]:
print("Joining train set with an additional store information.")

# by specifying inner join we make sure that only those observations 
# that are present in both train and store sets are merged together
train_store = pd.merge(train, store, how = 'inner', on = 'Store')

print("In total: ", train_store.shape)
train_store.head()

### 1.5 Store types

In this section we will closely look at different levels of `StoreType` and how the main metric `Sales` is distributed among them.  

In [ ]:
train_store.groupby('StoreType')['Sales'].describe()

`StoreType` B has the highest average of Sales among all others, however we have much less data for it. So let's print an overall sum of `Sales` and `Customers` to see which `StoreType` is the most selling and crowded one:

In [ ]:
train_store.groupby('StoreType')['Customers', 'Sales'].sum()

Clearly stores of type A. `StoreType` D goes on the second place in both `Sales` and `Customers`.
What about date periods?

#### 1.5.1 Impact of Promotion by store type

Seaborn's plots are an excellent tool for this task. Instead of showing a scatter plot, lets do more than ploting the data by estimating the central tendency and a confidence interval for that estimate.

In [ ]:
# Execution: 15 sec
sns.relplot(data = train_store, x = 'Month', y = 'Sales',
            kind='line', hue='Promo', col='StoreType', col_order= ['a','b','c','d'])

All store types follow the same trend but at different scales depending on the presence of the (first) promotion `Promo` and `StoreType` itself (case for B).

__Already at this point, we can see that Sales escalate towards Christmas holidays. But we'll talk about seasonalities and trends later in the Time Series Analysis section.__

In [ ]:
# Execution: 15 sec
sns.relplot(data = train_store, x = 'Month', y = "SalePerCustomer",
            kind='line', hue='Promo', col='StoreType', col_order= ['a','b','c','d'])

Aha! Eventhough the plots above showed `StoreType` B as the most selling and performant one (and the one with the highest variability), in reality it is not true. The highest `SalePerCustomer` amount is observed at the `StoreType` D, about \\$12 with `Promo` and \\$10 without. As for `StoreType` A and C it is about \\$9. 

Low `SalePerCustomer` amount for `StoreType` B describes its Buyer Cart: there are a lot of people who shop essentially for "small" things (or in a little quantity). Plus we saw that overall this `StoreType` generated the least amount of sales and customers over the period.

#### 1.5.1 Impact of day of the week by store type

In [ ]:
# Execution: 20 sec
sns.relplot(data = train_store, x = 'Month', y = "Sales",
            kind='line', hue='DayOfWeek', col='StoreType', col_order= ['a','b','c','d'], legend='full')

We see that stores of `StoreType` C sell similarly over the entire week, whereas others have a lot of variability.

To complete our preliminary data analysis, we can add variables describing the **number of months** during which competition and promotion were opened:

In [ ]:
# competition open time (in months)
train_store['CompetitionOpen'] = 12 * (train_store.Year - train_store.CompetitionOpenSinceYear) + \
        (train_store.Month - train_store.CompetitionOpenSinceMonth)
    
# Promo open time
train_store['PromoOpen'] = 12 * (train_store.Year - train_store.Promo2SinceYear) + \
        (train_store.WeekOfYear - train_store.Promo2SinceWeek) / 4.0

# replace NA's by 0
train_store.fillna(0, inplace = True)

# average PromoOpen time and CompetitionOpen time per store type
train_store.loc[:, ['StoreType', 'Sales', 'Customers', 'PromoOpen', 'CompetitionOpen']].groupby('StoreType').mean()

The most selling and crowded `StoreType` A doesn't appear to be the one the most exposed to competitors. Instead it's a `StoreType` B, which also has the longest running period of promotion.

### 1.6 Correlational Analysis

We are finished with adding new variables to the data, so now we can check the overall correlations by plotting a `seaborn` heatmap:

In [ ]:
# Compute the correlation matrix 
# exclude 'Open' variable
corr_all = train_store.drop('Open', axis = 1).corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr_all, dtype = np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize = (11, 9))

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr_all, mask = mask,
            square = True, linewidths = .5, ax = ax, cmap = "BuPu")      
plt.show()

As mentioned before, we have a strong positive correlation between the amount of Sales and Customers of a store. We can also observe a positive correlation between the fact that the store had a running promotion (`Promo` equal to 1) and amount of `Customers`. 

However, as soon as the store continues a consecutive promotion (`Promo2` equal to 1) the number of `Customers` and `Sales` seems to stay the same or even decrease, which is described by the pale negative correlation on the heatmap. The same negative correlation is observed between the presence of the promotion in the store and the day of a week.

In [ ]:
# sale per customer trends
sns.relplot(data = train_store, x = 'DayOfWeek', y = "Sales",
            kind='line', hue='Promo2', col='Promo')


There are several things here:
- In case of no promotion, both `Promo` and `Promo2` are equal to 0, `Sales` tend to peak on Sunday (!). Though we should note that `StoreType` C doesn't work on Sundays. So it is mainly data from `StoreType` A, B and D.
- On the contrary, stores that run the promotion tend to make most of the `Sales` on Monday. This fact could be a good indicator for Rossmann marketing campaigns. The same trend follow the stores which have both promotion at the same time (`Promo` and `Promo2` are equal to 1).
- `Promo2` alone doesn't seem to be correlated to any significant change in the `Sales` amount. This can be also prooved by the blue pale area on the heatmap above.

### 1.7 Conclusion of EDA

- The most selling and crowded `StoreType` is A.


- The best "Sale per Customer" `StoreType` D indicates to the higher Buyer Cart. We could also assume that the stores of this types are situated in the rural areas, so that customers prefer buying more but less often.


- Low `SalePerCustomer` amount for `StoreType` B indicates to the possible fact that people shop there essentially for small things. Which can also indicate to the label of this store type - "urban" - as it's more accessible for public, and customers don't mind shopping there from time to time during a week.


- Customers tends to buy more on Mondays when there's one promotion running (`Promo`) and on Sundays when there is no promotion at all (both `Promo` and `Promo1` are equal to 0).


- Promo2 alone doesn't seem to be correlated to any significant change in the `Sales` amount.

<br>
# 2. Time Series Components

What makes a time series different from a regular regression problem? 

- It is time dependent. The basic assumption of a linear regression that the observations are independent doesn’t hold in this case.


- Along with an increasing or decreasing trend, most time series have some form of seasonality trends, i.e. variations specific to a particular time frame. For example, for Christmas holidays, which we will see in this dataset.

<div class = "alert alert-block alert-info"> We build a time series analysis on store types instead of individual stores. The main advantage of this approach is its simplicity of presentation and overall account for different trends and seasonaltities in the dataset. </div>

In this section, we will analyse time series data: its trends, sesonalities and autocorrelation. Usually at the end of the analysis, we are able to develop a seasonal ARIMA (Autoregression Integrated Moving Average) model but it won't be our main focus today. Instead, we try to understand the data, and only later come up with the forecasts using Prophet methodology.

### 2.1 Seasonality

We take four stores from store types to represent their group:
- Store number 2 for `StoreType` A
- Store number 85 for `StoreType` B, 
- Store number 1 for `StoreType` C 
- Store number 13 for `StoreType` D. 

Only for visualization purposes it makes sense to increase **granularity** to weekly level (in pandas jargon this would be to *downsample the data from days to weeks*) using the `resample` method to see the present trends more clearly.

Lets plot the four sample stores:

In [ ]:
# preparation: input should be float type
train['Sales'] = train['Sales'] * 1.0

# store types
sales_a = train[train.Store == 2]['Sales']
sales_b = train[train.Store == 85]['Sales'].sort_index(ascending = True) # solve the reverse order
sales_c = train[train.Store == 1]['Sales']
sales_d = train[train.Store == 13]['Sales']

f, (ax1, ax2, ax3, ax4) = plt.subplots(4, figsize = (12, 13))

# store types
sales_a.resample('W').sum().plot(color = c, ax = ax1).set_title("Store Type A (2)")
sales_b.resample('W').sum().plot(color = c, ax = ax2).set_title("Store Type B (85)")
sales_c.resample('W').sum().plot(color = c, ax = ax3).set_title("Store Type C (1)")
sales_d.resample('W').sum().plot(color = c, ax = ax4).set_title("Store Type D (13)")

plt.subplots_adjust(hspace=0.5)
plt.show()

Retail sales for `StoreType` A and C tend to peak for the Christmas season and then decline after the holidays. We might have seen the same trend for `StoreType` D (at the bottom) but there is no information from July 2014 to January 2015 about these stores as they were closed.

### 2.2 Yearly trend

The next thing to check the presence of a trend in series. Lets do a time series decomposition and extract the Cycle-Trend (just Trend for simplicity) component.

In [ ]:
f, (ax1, ax2, ax3, ax4) = plt.subplots(4, figsize = (12, 13))

# monthly
decomposition_a = seasonal_decompose(sales_a, model = 'additive', freq = 365)
decomposition_a.trend.plot(color = c, ax = ax1).set_title("Store Type A (2) - Trend")

decomposition_b = seasonal_decompose(sales_b, model = 'additive', freq = 365)
decomposition_b.trend.plot(color = c, ax = ax2).set_title("Store Type B (85) - Trend")

decomposition_c = seasonal_decompose(sales_c, model = 'additive', freq = 365)
decomposition_c.trend.plot(color = c, ax = ax3).set_title("Store Type C (1) - Trend")

decomposition_d = seasonal_decompose(sales_d, model = 'additive', freq = 365)
decomposition_d.trend.plot(color = c, ax = ax4).set_title("Store Type D (13) - Trend")

plt.subplots_adjust(hspace=0.7)
plt.show()

Overall sales seems to increase, however not for the `StoreType` C (a third from the top). Eventhough the `StoreType` A is the most selling store type in the dataset, it seems that it can follow the same decresing trajectory as `StoreType` C did.

We can do the same decomposition for the seasonal component and inspect visually, however, its easier to find seasonal patterns by looking at autocorrelation plots.

### 2.3 Autocorrelation

The next step in our time series analysis is to review Autocorrelation Function (ACF) and Partial Autocorrelation Function (PACF) plots. 

ACF is a measure of the correlation between the timeseries with a lagged version of itself. For instance at specific lag 5, ACF would compare series at time instant $t_{1}...t_{n}$ with series at instant $t_{1-5}...t_{n-5}$.

PACF, on the other hand, measures the correlation between the timeseries with a lagged version of itself but after eliminating the variations explained by the intervening comparisons. Eg. at lag 5, it will check the correlation but remove the effects already explained by lags 1 to 4. 

In [ ]:
# acf and pacf for A
print("ACF and PACF for A (until lag 50)")
# figure for subplots
plt.figure(figsize = (12, 8))
plt.subplot(421); plot_acf(sales_a, lags = 50, ax = plt.gca(), color = c)
plt.subplot(422); plot_pacf(sales_a, lags = 50, ax = plt.gca(), color = c)
plt.show()

# acf and pacf for B
print("ACF and PACF for B (until lag 50)")
# figure for subplots
plt.figure(figsize = (12, 8))
plt.subplot(423); plot_acf(sales_b, lags = 50, ax = plt.gca(), color = c)
plt.subplot(424); plot_pacf(sales_b, lags = 50, ax = plt.gca(), color = c)
plt.show()

# acf and pacf for C
print("ACF and PACF for C (until lag 50)")
# figure for subplots
plt.figure(figsize = (12, 8))
plt.subplot(425); plot_acf(sales_c, lags = 50, ax = plt.gca(), color = c)
plt.subplot(426); plot_pacf(sales_c, lags = 50, ax = plt.gca(), color = c)
plt.show()

# acf and pacf for D
# figure for subplots
plt.figure(figsize = (12, 8))
print("ACF and PACF for D (until lag 50)")
plt.subplot(427); plot_acf(sales_d, lags = 50, ax = plt.gca(), color = c)
plt.subplot(428); plot_pacf(sales_d, lags = 50, ax = plt.gca(), color = c)
plt.show()



We can read these plots horizontally. Each horizontal pair is for one `StoreType`, from A to D. In general, those plots are showing the correlation of the series with itself, lagged by $x$ time units correlation of the series with itself, lagged by $x$ time units.

There are two things common for each pair of plots: non randomnes of the time series and high $lag 1$ (which will probably need a higher order of differencing d/D).

- Type A and type B:
Both types show seasonalities at certain lags. For type A, it is each 12th observation with positives spikes at the 12 (s) and 24(2s) lags and so on. For type B it's a weekly trend with positives spikes at the 7(s), 14(2s), 21(3s) and 28(4s) lags. 


- Type C and type D:
Plots of these two types are more complex. It seems like each observation is coorrelated to its adjacent observations. 

<font color=red>**EXERCISE: If we were to fit a univariate linear regression model with only one lagged variable for store type B, which lag would I use?**</font>

<br>
# 3. Statistical Model Forecasting

In this section we will perform univariate forecasting for the next 6 weeks for only one store (Store 1 in Store Type C) using the Facebook Prophet package. The Core Data Science team at Facebook published in 2017 a new procedure for forecasting time series data called [Prophet](https://research.fb.com/prophet-forecasting-at-scale/). It is based on an additive model where non-linear trends are fit with yearly and weekly seasonality, plus holidays.

$$
y_t = g_t + s_t + h_t + \epsilon_t
$$

Here $g_t$ is the trend function which models non-periodic changes in the value of the time series, $s_t$ represents periodic changes (e.g., weekly and yearly seasonality), and $h_t$  represents the effects of holidays which occur on potentially irregular schedules over one or more days. The error term $\epsilon_t$ represents any idiosyncratic changes which are not
accommodated by the model; later we will make the parametric assumption that $\epsilon_t$
is normally distributed

Lets reload the data as this kind of model is **univariate** (with the exception of the holiday term), so we cannot use external factors coming from store characteristics or promotions.

In [ ]:
# importing data
df = pd.read_csv("data/train.csv",  
                    low_memory = False)

# remove closed stores and those with no sales
df = df[(df["Open"] != 0) & (df['Sales'] != 0)]

# sales for the store number 1 (StoreType C)
sales = df[df.Store == 1].loc[:, ['Date', 'Sales']]

# reverse to the order: from 2013 to 2015
sales = sales.sort_index(ascending = False)

# to datetime64
sales['Date'] = pd.DatetimeIndex(sales['Date'])
sales.dtypes

In [ ]:
# from the prophet documentation every variables should have specific names
sales = sales.rename(columns = {'Date': 'ds',
                                'Sales': 'y'})
sales.head()

In [ ]:
# plot daily sales
ax = sales.set_index('ds').plot(figsize = (12, 4), color = c, title = 'Daily Sales for Store 1 (Type A)')
ax.set_ylabel('Daily Number of Sales')
ax.set_xlabel('Date')
plt.show()

### 3.1 Modeling with Holidays

Prophet allows to [model for holidays](https://facebookincubator.github.io/prophet/docs/holiday_effects.html), and that's what we do here.

The `StateHoliday` variable in the dataset indicates a state holiday, at which all stores are normally closed. There are also `SchoolHoliday` in the dataset at which ceratin stores are also closing their doors.

In [ ]:
# create holidays dataframe
state_dates = df[(df.StateHoliday == 'a') | (df.StateHoliday == 'b') & (df.StateHoliday == 'c')].loc[:, 'Date'].values
school_dates = df[df.SchoolHoliday == 1].loc[:, 'Date'].values

state = pd.DataFrame({'holiday': 'state_holiday',
                      'ds': pd.to_datetime(state_dates)})
school = pd.DataFrame({'holiday': 'school_holiday',
                      'ds': pd.to_datetime(school_dates)})

holidays = pd.concat((state, school))      

holidays.head(3).append(holidays.tail(3))

In [ ]:
#Execution: 55 seg
start = timer()

# set the uncertainty interval to 95% (the Prophet default is 80%)
my_model = Prophet(interval_width = 0.95, 
                   holidays = holidays)
my_model.fit(sales)

# dataframe that extends into future 6 weeks 
future_dates = my_model.make_future_dataframe(periods = 6*7)

end = timer()
print(end - start) 

print("First week to forecast.")
future_dates.tail(7)

The above fitting procedure (training) took 50 seconds for only 1 store. We have 1115, so thats over 15 hrs to fit all stores (and we are only working with a few million rows).

In [ ]:
# Execution 1:30 min

# predictions
forecast = my_model.predict(future_dates)

# preditions for last week
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(7)

The forecast object here is a new dataframe that includes a column $\hat{y}$ with the forecast, as well as columns for components and uncertainty intervals.

In [ ]:
fc = forecast[['ds', 'yhat']].rename(columns = {'Date': 'ds', 'Forecast': 'yhat'})

Prophet plots the observed values of our time series (the black dots), the forecasted values (blue line) and the uncertainty intervals of our forecasts (the blue shaded regions).

In [ ]:
# visualizing predicions
my_model.plot(forecast);

As we see Prophet catches the trends and most of the time gets future values right.

One other particularly strong feature of Prophet is its ability to return the components of our forecasts. This can help reveal how daily, weekly and yearly patterns of the time series plus manyally included holidayes contribute to the overall forecasted values:

In [ ]:
# Execution 1:40 min
my_model.plot_components(forecast);

- The first plot shows that the monthly sales of store number 1 has been linearly decreasing  over time
- The second plot shows the holiday gaps included in the model. 
- The third plot highlights the fact that the weekly volume of last week sales peaks towards the Monday of the next week.
- The fourth plot shows that the most buzy season occurs during the Christmas holidays.

<div class = "alert alert-block alert-info"> If we want to model seasonality, then we need at least two times the seasonal period to be able to model it (preferably more), otherwise your model has no way of knowing whether a spike is a seasonal variation or just a one time impulse. So for example to model a weekly seasonality, you need at least 14 days of training data (plus whatever you will use for testing), and for a yearly seasonality, you will need at least 730 days of data, etc.... </div>

### 3.2 Time Series Cross Validation (Walk Forward Testing)

We are interested in testing the generalization of the model. The evaluation happens on a rolling forecasting fashion to capture temporal effects. We want to be able to predict unseen future data.

![Backtesting](img/cross_validation.png)

In most time series cross validation methods we need to chose an *initial training period*, a *cutoff* that will move on each fitting (which shifts forward by the *period*) and a forecasting horizon for each iteration.

In [ ]:
# Execution: 3 min
# To simplify given time constraints we just test the last week.
df_cv = cross_validation(my_model, initial='928 days', period='7 days', horizon = '7 days')
df_cv.head()

We will calculate some statistics such as mean squared error (MSE), root mean squared error (RMSE), mean absolute error (MAE) and mean absolute percent error (MAPE). These are averages over **all forecasting horizons**.

In [ ]:
df_p = performance_metrics(df_cv)
df_p

### 3.3 Conclusions for Univariate forecasting

During this part we discussed time series analysis with `.seasonal_decompose()`, `ACF` and `PCF` plots and fited forecasting model using a relatively new procedure by Facebook `Prophet`.

We can now present main advantages and drawbacks of univariate time series forecasting:

__Advantages__
- Powerfull tool for the time series forecasting as it accounts for time dependencies, seasonalities and holidays (Prophet: manually).
- Easily implemented with R `auto.arima()` from `forecast` package, which runs a complex grid search and sophisticated algorithm behind the scene.

__Drawbacks__
- Doesn't catch interactions between external features, which could improve the forecasting power of a model. In our case, these variables are `Promo` and `CompetitionOpen`. 
- Fitting seasonal ARIMA model needs at least 4 whole seasons in the dataset, which can be the the biggest drawback for new companies.
- Seasonal ARIMA in Python has 7 hyperparameters which can be tuned only manually affecting significantly the speed of the forecasting process (Use `forecast` package in **R** for univariate forecasting!).
- Statistical methods are generally too slow for large scale problems. In Retail for example we need to forecast millions of time series so classical methods are difficult to implement.

<br>
## 4. Machine Learning Forecasting

In this section we will be testing machine learning algorithms, specifically [LightGBM](https://github.com/Microsoft/LightGBM) an implementation of Gradient Boosted Decision trees designed for speed and performance. Its a type of [Regularized Gradient Boosting](http://datascience.la/xgboost-workshop-and-meetup-talk-with-tianqi-chen/), as it uses a more regularized model formalization to control over-fitting. 

Additional advantages of this algorithm are:

- Automated handling of missing values: XGB uses a "learned" default direction for the missing values. "Learned" means learned in the tree construction process by choosing the best direction that optimizes the training loss.
- Interactive feature analysis: plots the structure of decision trees with splits and leaves.
- Feature importance analysis: a sorted barplot of the most significant variables.

It has Faster training speed and better accuracy compared to [XGBoost](https://github.com/dmlc/xgboost). It also capable of handling large-scale data.

<div class = "alert alert-block alert-info"> The *Automated handling of missing values* feature in tree based methods is **VERY** useful because it enables us to predict on unseen categorical observations. For example **NEW** products in case SKU is part of the forecast granularity.</div>

ML algorithms are good for learning across time series what we called **Cross Learning **. Given the nature of the Rossmann problem, we expect to get good results as the model will learn from different store patterns.

### 4.1 Data Encoding
Many Machine Learning models do not support anything else than numerical features (such as XGBoost). So prior to modeling we need to encode certain factor variables into numerical plus extract dates as we did before for the train set.

In [ ]:
# to numerical
mappings = {'0':0, 'a':1, 'b':2, 'c':3, 'd':4}
train_store.Assortment.replace(mappings, inplace = True)
train_store.StoreType.replace(mappings, inplace = True)
train_store.StateHoliday.replace(mappings, inplace = True)
train_store.drop('PromoInterval', axis = 1, inplace = True)
train_store['StateHoliday'] = train_store.StateHoliday.astype(float)
train_store['Assortment'] = train_store.Assortment.astype(float)

### 4.2 Time Series Feature Enginering

What if we incorporate the AR (Auto Regressive) and MA (Moving Average) concepts into the feature engineering process? 
Traditional methods apply this idea to the response only (the output) but we can also incorporate this into our predictors. This process is known as **expanding the feature space** because we are creating new features from the existing predictor space.

- #### Lagged Values
Lag features are the classical way that time series forecasting problems are transformed into supervised learning problems. 
The simplest approach is to predict the value at the next time $t+1$ given the value at the previous time $t-1$.

- #### Rolling Window Statistics
A step beyond adding raw lagged values is to add a summary of the values at previous time steps. We can calculate summary statistics across the values in a sliding window and include these as features in our dataset. Perhaps the most useful is the mean of the previous few values, also called the rolling mean. For example, we can calculate the mean of the previous two values and use that to predict the next value. For the temperature data, we would have to wait 3 time steps before we had 2 values to take the average of before we could use that value to predict a 3rd value.


#### Opex Analytics Framework

Opex has a forecasting framework for the time series feature engineering process inspired by the work done by H2O in their [Driverless AI Forecasting solution](http://docs.h2o.ai/driverless-ai/latest-stable/docs/userguide/time-series.html). Lets import the module from [github](https://github.com/opex-analytics/opex_lagged_feature_generator).

In [ ]:
import opex_lagged_feature_generator
from opex_lagged_feature_generator.lag_pattern import LagPattern
from opex_lagged_feature_generator.lagged_feature_generator import LaggedFeatureGenerator

Lets save the date as an index for ordering purposes.

In [ ]:
train_store.index = train_store['Date']

Lets generate lagged features and rolling windows means for the `Sales` column grouped by `Store`.

In [ ]:
try:
    series_names = ['Sales'] # list of series to lag
    order_by = 'Date'  # columns to order data by
    partition_by = ['Store']        # columns to identify unique time series in data

    lag_patterns = list()
    lag_patterns.append(LagPattern(5)) # shifted series lagged 5 units

    # Explanation of Lag_Pattern constructor
    lag_patterns.append(LagPattern(min_max_lag=(1, 5),  # boundaries -- (start, stop) from python range function: range([start], stop[, step]))
                                   step_size=2,  # frequency of lag features -- step from python range function: range([start], stop[, step])
                                   window_size=2,  # window over which to aggregate before lagging
                                   create_delta_features=True,  # boolean indicating whether to create trend features (difference between adjacent lags)
                                   window_aggregation_funcs='mean')) # single/list of functions that can be applied to a pd.Series


    # Explanation of Lagged_Predictor_Generator constructor
    lpg = LaggedFeatureGenerator(df, # dataframe to perform feature generation on
                                     lag_patterns=lag_patterns, # single/list of Lag_Pattern objects
                                     series_names=series_names, # single/list/dict of series columns (what you want to lag)
                                     order_by=order_by, # single/list of columns by which to order the dataframe
                                     partition_by=partition_by, # OPTIONAL single/list of columns by which to groupby prior to lagging
                                     order_df=None) # OPTIONAL argument if you want to filter by some specific order_by values

    res = lpg.execute() # create lagged features
except Exception:
    pass

### 4.3 Forecast of Predictors

In many scenarios we will need forecast of the predictors.  For example, if I train my model using weather measures such as daily temperature, I will need this feature available when doing predictions for next week.
In other cases, the data is not simply not available, this is the case for the `Customers` feature in the training set which is missing from the test set. To simplify this notebook, we will drop this feature but in reality, we could forecast this predictor using univariate models for example.

In [ ]:
# Drop some columns to simplify exercise
try:
    train_store = train_store.drop(['Customers'], axis=1) 
    train_store = train_store.drop(['SalePerCustomer'], axis=1) 
    train_store = train_store.drop(['Date'], axis=1)
    test_store = test_store.drop(['Id'], axis=1)
except Exception:
    pass

### 4.4 Model Training and Prediction on a NEW store

Future sessions will be focused on training techniques, hyperparameters tunning, model comparisson and evaluation.

In this section I want to train my model with *ALL* available data and try to predict sales for a *NEW* store. We will remove the *Store 1* from the training dataset and evaluate Store 1 historical Sales using our ML model predictions.

Lets remove **Store 1** from the dataset:

In [ ]:
train_store_wo_1 = train_store[train_store.Store != 1]


And save historical sales from **Store 1**:

In [ ]:
store_1 = train_store[train_store.Store == 1]

Create the train and validation datasets

In [ ]:
train = train_store_wo_1.iloc[train_store_wo_1.index <= '2015-07-26']
validation = train_store_wo_1.iloc[train_store_wo_1.index > '2015-07-26']


y_train = train['Sales']
X_train = train.drop(['Sales'], axis=1)

y_val = validation['Sales']
X_val = validation.drop(['Sales'], axis=1)

# create dataset for lightgbm
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train)

Train the model using some default hyperparameters:

In [ ]:
# specify your configurations as a dict
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

print('Starting training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=30,
                valid_sets=lgb_eval,
                early_stopping_rounds=5)

Predict on **Store 1**:

In [ ]:
print('Starting predicting...')
# prepare data for store 1
y_store_1 = store_1['Sales']
X_store_1 = store_1.drop(['Sales'], axis=1)
# predict
y_store_1_pred = gbm.predict(X_store_1, num_iteration=gbm.best_iteration)
# eval
print('The rmse of prediction is:', mean_squared_error(y_store_1, y_store_1_pred) ** 0.5)

Lets compare our predictions with the historical data:

In [ ]:
df = pd.DataFrame({'date': y_store_1.index,'prediction':y_store_1_pred, 'actual':y_store_1.values})
df = df.sort_values(by=['date'], ascending=True)

df.plot(x="date", y=["actual", "prediction"], figsize=(14,8))
plt.show()

Looks like a systematic bias, we are **over forecasting** since we dont have history of the level of the time series for this store.  From the peaks we can see that probably the model is at least capturing events such as promotions.

If we were able to train the model with some historical data for **Store 1** we would be able to capture this level and improve our forecasts!


<font color=red>**EXERCISE: Train Model with some historical data from Store 1**</font>


### 4.5 Evaluation Metric
The major challenge for grocery retailers is dealing with perishable goods like food. For Rossmann, **over-forecasting** is undesirable so having large errors in their forecast is unacceptable. This means that a metric such as $RMSE$ could be good match for Rossmann e.g. being off by 10 is more than twice as bad as being off by 5.

$$
RMSE=\sqrt{\frac{1}{n}(y^{(i)}-\hat{y}^{(i)})^{2}}
$$

However, Rossmann do not want to penalize all stores equally. Their main stores, the ones that sell more are a priority. These means we prefer accurate forecasts for larger stores. In this case, if we have 2 stores, and both of them have two observations with an Absolute Error of 4, this error could be considered “small” or “big” when we compare it to the actual sales value of that store. $MAPE$ in this case is prefered because it compares percentage wise.

$${\it MAPE} =\displaystyle \sqrt{
\frac{1}{n}\sum_{i=1}^{n}\mid\frac{y^{(i)}-\hat{y}^{(i)}}{y^{(i)}}\mid}$$

What do we do? Well we can just use a hybrid metric to handle both cases, the Root Mean Square Percentage Error ($RMSPE$):

$${\it RMSPE} =\displaystyle \sqrt{
\frac{1}{n}\sum_{i=1}^{n}(\frac{y^{(i)}-\hat{y}^{(i)}}{y^{(i)}})^{2}}$$

#### Train with same Evaluation Metric

In most cases, using custom loss functions like the one above is not direct. Luckily, **LightGBM** or **XGBoost** let users set the loss function as a parameter. However, thats not the case for all packaged algorithms.

In other cases a transformation of the response or output is needed. In this example, the model can still be trained using $RMSE$ by applying a log transformation to `Sales`. This is equivalent than minimizing $RMSPE$, see Annex for a proof. Once we perform the forecasting we will unwind log transformations in reverse order. 

### 4.6 Final thoughts

Time Series Analysis is a must for time series data. It goes much deeper than ad-hoc Exploratory Data Analysis, revealing trends, non randomness of the data and seasonalities. 

I was particularly excited to use a new forecasting procedure `Prophet`. Eventhough this tool is still under development, it has everything set for the advanced modeling as it can account for change points in trends and holidays in the data. In the meantime, the most sophisticated tool for the Time Series Analysis stays `auto.arima` from R `forecast` package. 

**Cross Learning**, usage of **external predictors**, **faster training time** and ability to better forecast unseen data are some of the benefits of using ML models.

## Annex
The scoring method was Root Mean Square Percent Error (RMSPE) and not Root Mean Square Loss (RMSE) which is the loss function used by default in many models. This means that we either have to transform our target variable or write our own loss function. 

It turns out that doing a simple log transformation of the target variable is theoretically sound as long as the difference between our target values and prediction values is small. The following shows that RMSPE is the same as RMSE of $\log$ transformed response variable:

$${\it RMSPE} =\displaystyle \sqrt{
\frac{1}{n}\sum_{i=1}^{n}(\frac{y^{(i)}-\hat{y}^{(i)}}{y^{(i)}})^{2}}$$

$$
RMSE=\sqrt{\frac{1}{n}(y^{(i)}-\hat{y}^{(i)})^{2}}
$$
In order to make RMSPE equal RMSE we need to find a function $f(y)$ that will make:

$$
f(y^{(i)})-f(\hat{y}^{(i)})= \displaystyle \frac{y^{(i)}-\hat{y}^{(i)}}{y^{(i)}}
     \qquad (1) $$ 


We can approximate $f(y)$ using Taylor Expansion, with the assumption that $\delta$ (the difference between our prediction and response variables) is small:

$$
\delta=y^{(i)}-\hat{y}^{(i)}
$$

$$
f(\hat{y}^{(i)})=f(y^{(i)}+\delta)\approx f(t)+f'(t)\sigma+...
$$

Drop the higher order terms of the Taylor Expansion and plugging into equation $(1)$ and simplifying:

$$
f(y^{(i)})+f'(y^{(i)})\delta-f(y^{(i)})\approx\ \frac{\hat{y}^{(i)}-y^{(i)}}{y^{(i)}}
$$
$$
f'(t)\delta\approx\ \frac{\delta}{t}
$$
$$
f'(t)\approx\ \frac{1}{t}
$$
$$
f(y)=\ln(y)+C
$$

This shows that the correct way to make RMSPE and RMSE approximately equal is to take the $\log$ transformation of our prediction and target variables.


# References

*This notebook is based on the work done by [Elena Petrova](https://github.com/elena-petrova).*

https://github.com/elena-petrova/rossmann_TSA_forecasts

http://cs229.stanford.edu/proj2015/193_report.pdf